In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_count_sla_id = 'select count(*) from sladb.dbo.tbl_ref_sla'

In [6]:
sql_distinct_sla_id = 'select distinct sla_id from sladb.dbo.tbl_ref_sla'

In [7]:
# sql_sla_code = '''select * from \
# sladb.dbo.tbl_ref_sla_code'''

In [8]:
sql_sla_code = '''select min(sla_code) as min_sla_code, 
                          max(sla_code) as max_sla_code, 
                          count(*) as n_sla_codes
                   from sladb.dbo.tbl_ref_sla_code'''

In [9]:
sql_date_category = 'select distinct(date_category_id) from sladb.dbo.tbl_ref_sla_season_category'

In [10]:
sql_sla_translation = 'select sla_id, sla_code, date_category_id from sladb.dbo.tbl_ref_sla_translation'

## Execute all of the sql queries and store the results

In [11]:
before_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [12]:
before_count_sla_id

8

In [13]:
before_squared_count_sla_id = before_count_sla_id * before_count_sla_id

In [14]:
before_squared_count_sla_id

64

In [15]:
before_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [16]:
before_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,E
5,F
6,N
7,T


In [17]:
before_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [18]:
before_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,64,64


In [19]:
before_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [20]:
before_date_category

,date_category_id
0,1
1,2


In [21]:
before_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [22]:
before_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,17,1
...,...,...,...
123,T,28,2
124,T,30,2
125,T,36,2
126,T,47,2


In [255]:
timing = []

In [256]:
descriptions = []

In [257]:
results = []

## Before Tests

### First before test

In [258]:
timing.append('before')

In [259]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of 
                       sla_codes in tbl_ref_sla_code''')

In [260]:
results.append(before_squared_count_sla_id == before_sla_code['n_sla_codes'].values[0])

### Second before test

In [261]:
timing.append('before')

In [262]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code
                       in tbl_ref_sla_code''')

In [263]:
results.append(before_sla_code['n_sla_codes'].values[0] == before_sla_code['max_sla_code'].values[0])

### Third before test

In [264]:
timing.append('before')

In [265]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [266]:
results.append(1 == before_sla_code['min_sla_code'].values[0])

### Fourth before test

In [267]:
timing.append('before')

In [268]:
descriptions.append('''Test that the count of records in tbl_ref_sla_translation is equal to the 
                       count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids 
                       in tbl_ref_sla_season_category''')

In [269]:
results.append(len(before_sla_translation) == (before_sla_code['n_sla_codes'].values[0] * len(before_date_category)))

### Fifth before test

In [270]:
timing.append('before')

In [271]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [272]:
before_sla_translation_nodup = before_sla_translation.drop_duplicates()

In [273]:
results.append(len(before_sla_translation) == len(before_sla_translation_nodup))

### Sixth before test

In [274]:
timing.append('before')

In [275]:
descriptions.append(
    '''Test that all sla_ids, sla_codes and date_category_ids have been \
    accounted for in tbl_ref_sla_translation''')

In [276]:
before_unique_sla_translation = before_sla_translation.nunique()

In [277]:
results.append(
    ((before_unique_sla_translation['sla_id'] == before_count_sla_id)
     and (before_unique_sla_translation['date_category_id'] == len(
         before_date_category))
     and (before_unique_sla_translation['sla_code'] ==
          before_sla_code['n_sla_codes'].values[0])))

## Insert the sla_id values into tbl_ref_sla so that the after tests can be conducted

In [278]:
ninserts = 2

In [279]:
sla_id = [x for x in string.ascii_uppercase if x not in(before_distinct_sla_id.values)]

In [280]:
sla_inserts = [[x, 'Testing new SLA: '+ x, 1, 2] for x in sla_id]

In [281]:
sla_inserts_df = pd.DataFrame(sla_inserts[0:ninserts], 
                              columns = ['sla_id', 'sla_desc', 'sla_min_days', 'sla_max_days'])

In [282]:
sla_inserts_df

,sla_id,sla_desc,sla_min_days,sla_max_days
0,G,Testing new SLA: G,1,2
1,H,Testing new SLA: H,1,2


In [283]:
# sla_inserts_df.to_sql('tbl_ref_sla', con = engine, index = False, if_exists = 'append')

## Execute all of the sql queries AFTER INSERT and store the results

In [284]:
after_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [285]:
after_count_sla_id

10

In [286]:
after_squared_count_sla_id = (after_count_sla_id)**2 

In [287]:
after_squared_count_sla_id

100

In [288]:
after_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [289]:
after_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,E
5,F
6,G
7,H
8,N
9,T


In [290]:
after_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [291]:
after_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,100,100


In [292]:
after_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [293]:
after_date_category

,date_category_id
0,1
1,2


In [294]:
after_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [295]:
after_sla_translation

,sla_id,sla_code,date_category_id
0,A,1,1
1,A,2,1
2,A,3,1
3,A,4,1
4,A,17,1
...,...,...,...
195,T,36,2
196,T,47,2
197,T,62,2
198,T,79,2


## After Tests

### First after test

In [296]:
timing.append('after')

In [297]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of 
                       sla_codes in tbl_ref_sla_code''')

In [298]:
results.append(after_squared_count_sla_id == after_sla_code['n_sla_codes'].values[0])

In [299]:
results

[True, True, True, True, True, True, True]

### Second after test

In [300]:
timing.append('after')

In [301]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code \
is equal to the maximum sla_code in tbl_ref_sla_code''')

In [302]:
results.append(after_sla_code['n_sla_codes'].values[0] == after_sla_code['max_sla_code'].values[0])

### Third after test

In [303]:
timing.append('after')

In [304]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [305]:
results.append(1 == after_sla_code['min_sla_code'].values[0])

### Fourth after test

In [306]:
timing.append('after')

In [307]:
descriptions.append(
    '''Test that the count of records in tbl_ref_sla_translation is \
    equal to the count of sla_codes in tbl_ref_sla_code multiplied by \
    the number of date_category_ids in tbl_ref_sla_season_category''')

In [308]:
results.append(len(after_sla_translation) == (after_sla_code['n_sla_codes'].values[0] * len(after_date_category)))

### Fifth before test

In [309]:
timing.append('after')

In [310]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [311]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [312]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Sixth after test

In [313]:
timing.append('after')

In [314]:
descriptions.append(
    '''Test that all sla_ids, sla_codes and date_category_ids have been \
    accounted for in tbl_ref_sla_translation''')

In [315]:
after_unique_sla_translation = after_sla_translation.nunique()

In [316]:
results.append(
    ((after_unique_sla_translation['sla_id'] == after_count_sla_id)
     and (after_unique_sla_translation['date_category_id'] == len(
         after_date_category)) and (after_unique_sla_translation['sla_code'] ==
                                    after_sla_code['n_sla_codes'].values[0])))

### Seventh after test

In [317]:
results

[True, True, True, True, True, True, True, True, True, True, True, True]

In [318]:
timing.append('after')

In [319]:
m = ninserts
n = before_count_sla_id

In [320]:
descriptions.append(
    '''Test that the number of rows insrted into tbl_ref_sla_code is equal to (m+n)**2-n**2 '''
)

In [321]:
results.append(after_sla_code.n_sla_codes.values[0] -
               before_sla_code.n_sla_codes.values[0] == (m + n)**2 - n**2)

In [322]:
descriptions.append(
    '''Test that after removing original rows from tbl_ref_sla_trnaslation there are (m+n)**2 -(n)**2 rows leftover'''
)

### Eighth after test

In [323]:
timing.append('after')

In [324]:
descriptions.append(
    '''Test that the nmuber of rows in tbl_ref_sla_translation is equal to 2*(m+n)**2 '''
)

In [325]:
num_date_category_ids = len(after_date_category)

In [326]:
results.append(len(after_sla_translation) == num_date_category_ids * (m + n)**2)

### Ninth after test

In [327]:
timing.append('after')

In [328]:
descriptions.append('''After removing the original rows, \
test the uniqueness of the rows in tbl_ref_sla_translation''')

In [329]:
after_sla_translation_nodup = after_sla_translation.drop_duplicates()

In [330]:
results.append(len(after_sla_translation) == len(after_sla_translation_nodup))

### Tenth after test

In [331]:
timing.append('after')

In [332]:
descriptions.append(
    '''Ensuring the correct number of rows was inserted into tbl_ref_sla_trnalsation'''
)

In [333]:
added_rows_df = pd.merge(after_sla_translation,
         before_sla_translation,
         on=['sla_id', 'sla_code', 'date_category_id'],
         indicator=True,
         how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [334]:
results.append(len(added_rows_df)==(len(after_sla_translation) - len(before_sla_translation)))

## List of tuples with final test results

In [335]:
list(zip(timing, descriptions, results))

[('before',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of \n                       sla_codes in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code\n                       in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the minimum sla_code in tbl_ref_sla_code is equal to 1',
  True),
 ('before',
  'Test that the count of records in tbl_ref_sla_translation is equal to the \n                       count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids \n                       in tbl_ref_sla_season_category',
  True),
 ('before',
  'Test the uniqueness of the rows in tbl_ref_sla_translation',
  True),
 ('before',
  'Test that all sla_ids, sla_codes and date_category_ids have been     accounted for in tbl_ref_sla_translation',
  True),
 ('after',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of 